# A Step-by-Step Guide to HoloFusion example part 2/4 
  #  (accu implementation)


# Setup
 Firstly, we import all the module from Holofusion that we will use.

In [1]:
from holofusion import HoloFusion, HoloFusionSession
from time import time as t

##   Initialization
In this part, we create the HoloFusion and Session object that we will use for this example.

In [2]:
        holo_obj = HoloFusion(algorithm=2, training_data=1, multiple_weights=0)
        session = HoloFusionSession("Session", holo_obj)
        fx = open('execution_time.txt', 'w')
        

/home/james/anaconda/envs/python2.7/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1576: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input from file
Test data will be read using the Session's ingestor.
After ingesting the test data will be loaded into MySQL tables along with entries in the a metadata table.

In [3]:
        list_time = []
        start_time = t()

        session.ingest_dataset("data/clean_flight2/flight-data-test.csv")
        d = t()-start_time
        list_time.append(d)
        fx.write('ingest csv time: '+str(d)+'\n')
        print 'ingest csv time: '+str(d)+'\n'
        
        start_time = t()
        session.adding_training_data("data/clean_flight2/flight-data_training.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('adding training data: ' + str(d) + '\n')
        print 'adding training time: ' + str(d) + '\n'
        
        print 'Clean table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_clean", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()
        print 'Don\'t know table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_dk", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()

ingest csv time: 7.99743390083

['Source', 'Flight_Num', 'Scheduled_Dept', 'Actual_Dept', 'Scheduled_Arrival', 'Actual_Arrival']
adding training time: 11.0706410408

Clean table
+--------------------+--------------------+--------------+-----------+
|              Source|          Flight_Num|Scheduled_Dept|Actual_Dept|
+--------------------+--------------------+--------------+-----------+
|                ifly|2011-12-01-AA-384...|          6:00|       6:56|
|                  aa|2011-12-01-AA-466...|          6:00|       6:08|
|             flights|2011-12-01-AA-466...|          6:00|       6:08|
| businesstravellogue|2011-12-01-AA-466...|          6:00|       6:08|
|         flightstats|2011-12-01-UA-248...|         12:57|       1:32|
|           quicktrip|2011-12-01-UA-248...|         12:57|       1:32|
| businesstravellogue|2011-12-01-AA-205...|         10:40|      11:05|
|       flylouisville|2011-12-01-AA-205...|         10:40|      11:05|
|         flightstats|2011-12-01-AA-205..

## Featurization
We create the appropriate dictionary to run accu algorithm

In [4]:
        start_time = t()
        session.feature()
        d = t() - start_time
        list_time.append(d)
        fx.write('creating feature table time: '+str(d)+'\n')

        print ' feature table time: '+str(d)+'\n'

 feature table time: 2.61888098717



##  Learning
In the learning phase, we use the accu algorithm to get the most appropriate value for each object


In [5]:
        start_time = t()
        session.inference(iterations = 2000)
        d = t() - start_time
        list_time.append(d)
        fx.write('inference time: ' + str(d) + '\n')
        print 'inference time: ' + str(d) + '\n'
        

0
1000
Print weight_accuracy for the first 10 sources:
aa 0.99
helloflight 0.769183922046
boston 0.999589827728
weather 0.567702702703
airtravelcenter 0.693099273608
flightview 0.750209819555
flightstats 0.293577981651
usatoday 0.684241245136
ord 0.458333333333
panynj 0.779220779221
inference time: 371.294900179



## Evaluation
 In this part, we find the accuracy of our results by comparing them to ground truth for our data

In [6]:
        start_time = t()
        session.accuracy("data/clean_flight2/flight-data_truth.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('time to calculate accuracy: ' + str(d) + '\n')
        print 'time to calculate accuracy: ' + str(d) + '\n'
        fx.close()

['Flight_Num', 'Scheduled_Dept', 'Actual_Dept', 'Scheduled_Arrival', 'Actual_Arrival']
show the final table
+--------------------+-----------------+------------+
|            rv_index|          rv_attr|assigned_val|
+--------------------+-----------------+------------+
|2011-12-11-UA-843...|   Scheduled_Dept|        null|
|2011-12-12-UA-111...|   Actual_Arrival|        6:45|
|2011-12-18-AA-236...|      Actual_Dept|        null|
|2011-12-22-UA-642...|      Actual_Dept|        null|
|2011-12-15-UA-280...|      Actual_Dept|        null|
|2011-12-07-AA-241...|   Scheduled_Dept|        null|
|2011-12-16-UA-125...|   Scheduled_Dept|        null|
|2011-12-08-CO-443...|   Scheduled_Dept|        4:25|
|2011-12-20-CO-166...|Scheduled_Arrival|        6:00|
|2011-12-11-AA-149...|      Actual_Dept|        8:15|
|2011-12-10-UA-126...|   Scheduled_Dept|        9:58|
|2011-12-05-AA-197...|   Actual_Arrival|       10:43|
|2011-12-24-UA-616...|Scheduled_Arrival|        9:10|
|2011-12-10-AA-74-...|Schedu

Traceback (most recent call last):
  File "/home/james/anaconda/envs/python2.7/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/james/anaconda/envs/python2.7/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/home/james/anaconda/envs/python2.7/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/james/anaconda/envs/python2.7/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/home/james/anaconda/envs/python2.7/lib/python2.7/site-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/home/james/anaconda/envs/python2.7/lib/python2.7/site-packages/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError
